In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Neural Network
## Ivan Makaveev, 2MI0600203

In [2]:
seed = 1337
np.random.seed(seed)

mode = 'ALL'
activator = 0
hidden_count = 1
neurons_per_layer = 4
learning_rate = 0.1
loss = "BCE" # MSE or BCE

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_der(x):
    return sigmoid(x) * (1 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_der(x):
    return 1 - tanh(x) ** 2

In [4]:
def mse_loss_derivative(y, y_predicted):
    return -(y_predicted - y) # Derivative of Mean Squared Error

def bce_loss_derivative(y, y_predicted):
    return (y/y_predicted - (1-y)/(1-y_predicted)) # Derivative of Binary Cross Entropy

In [5]:
class Neuron:
    def __init__(self, input_size, activator, activator_der, learning_rate):
        self.input_size = input_size
        self.learning_rate = learning_rate
        self.activator = activator
        self.activator_der = activator_der
        
        self.weights = np.random.normal(0, np.sqrt(1 / input_size), size=input_size)
        self.bias = -1
        self.gradientW = 0
        self.gradientB = 0
        self.record = np.zeros(input_size)
        self.result = 0
        
    def forward(self, record):
        self.record = record
        self.result = np.dot(self.weights, record) + self.bias
        return self.activator(self.result)
    
    def backward(self, propagated_error):
        gradient = self.activator_der(self.result) * propagated_error
        self.gradientW = gradient * self.record
        self.gradientB = gradient
        return gradient
    
    def update(self):
        self.weights += self.learning_rate * self.gradientW
        self.bias += self.learning_rate * self.gradientB
        
    def predict(self, record):
        return self.activator(np.dot(self.weights, record) + self.bias)

class NeuronNetwork:
    def __init__(self, input_size, hidden_layers, neurons_per_layer, output_size, activator, lr, loss = "MSE"):
        self.hidden_layers = hidden_layers
        self.neurons_per_layer = neurons_per_layer
        self.output_size = output_size
        self.input_size = input_size
        self.activator = sigmoid if activator == 0 else tanh
        self.activator_der = sigmoid_der if activator == 0 else tanh_der
        self.lr = lr
        self.loss_der = mse_loss_derivative if loss == "MSE" else bce_loss_derivative
        
        self.layers = []
        isInputConnected = False
        for _ in range(hidden_layers):
            curr_layer = []
            for _ in range(neurons_per_layer):
                curr_layer.append(Neuron(neurons_per_layer if isInputConnected else input_size, self.activator, self.activator_der, self.lr))
            isInputConnected = True
            self.layers.append(curr_layer)
        
        curr_layer = []
        for _ in range(output_size):
            curr_layer.append(Neuron(neurons_per_layer if isInputConnected else input_size, self.activator, self.activator_der, self.lr))
        self.layers.append(curr_layer)

    def train(self, data, epochs):
        for _ in range(epochs):
            for x, y in data:
                layer_activations = [x]
                for layer in self.layers:
                    prev_act = layer_activations[-1]
                    curr_layer_act = np.array([neuron.forward(prev_act) for neuron in layer], dtype=np.float64)
                    layer_activations.append(curr_layer_act)

                y_predicted = layer_activations[-1]

                propagated_error = self.loss_der(y, y_predicted)
                for layer_idx in range(len(self.layers) - 1, -1, -1):
                    layer = self.layers[layer_idx]
                    prev_input_size = self.layers[layer_idx][0].input_size

                    grads = []
                    for i, neuron in enumerate(layer):
                        grad = neuron.backward(propagated_error[i])
                        grads.append(grad)

                    if layer_idx > 0:
                        propagated_prev = np.zeros(prev_input_size, dtype=np.float64)
                        for i, neuron in enumerate(layer):
                            propagated_prev += propagated_error[i] * neuron.weights
                        propagated_error = propagated_prev
                        
                    for neuron in layer:
                        neuron.update()
                        
    def predict(self, record):
        layer_activations = [record]
        for layer in self.layers:
            prev_act = layer_activations[-1]
            curr_layer_act = np.array([neuron.predict(prev_act) for neuron in layer], dtype=np.float64)
            layer_activations.append(curr_layer_act)
                
        return layer_activations[-1]

In [6]:
def get_bool_func(mode, inputs):
    if(mode == 'OR'):
        return [(i, i[0] | i[1])  for i in inputs]
    if(mode == 'AND'):
        return [(i, i[0] & i[1])  for i in inputs]
    if(mode == 'XOR'):
        return [(i, i[0] ^ i[1])  for i in inputs]
    raise ValueError("Unknown mode")
    
inputs = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]
)

In [7]:
def test_neural_net(mode):
    data = get_bool_func(mode, inputs)
    nn = NeuronNetwork(inputs.shape[1], hidden_count, neurons_per_layer, 1, activator, learning_rate, loss)
    nn.train(data, 5000)
    print(f"Mode: {mode}")
    for i in inputs:
        print(f"Input {i} -> {nn.predict(i)}")

In [8]:
if(mode == "ALL"):
    for m in ("AND", "OR", "XOR"):
        test_neural_net(m)
else:
    test_neural_net(mode)

Mode: AND
Input [0 0] -> [0.00036846]
Input [0 1] -> [0.00038172]
Input [1 0] -> [0.0003818]
Input [1 1] -> [0.99853552]
Mode: OR
Input [0 0] -> [0.00090942]
Input [0 1] -> [0.99968009]
Input [1 0] -> [0.99968009]
Input [1 1] -> [0.9996861]
Mode: XOR
Input [0 0] -> [0.00022307]
Input [0 1] -> [0.99582875]
Input [1 0] -> [0.99791442]
Input [1 1] -> [0.00810905]
